# Required Libraries

In [32]:
# Part A: Initial Setup and Imports

import pandas as pd
import numpy as np

# Load your datasets here, which would typically be in CSV or another file format:
# User gameplay data, deposit data, withdrawal data
# Assuming they are loaded as DataFrames:
# user_gameplay = pd.read_csv('user_gameplay.csv')
# deposit_data = pd.read_csv('deposit_data.csv')
# withdrawal_data = pd.read_csv('withdrawal_data.csv')


In [2]:
gameplay_data = pd.read_csv('D:/User_Gameplay.csv')
gameplay_data

,UserID,GamesPlayed,Datetime
0,851,1,01-10-2022 00:00
1,717,1,01-10-2022 00:00
2,456,1,01-10-2022 00:00
3,424,1,01-10-2022 00:00
4,845,1,01-10-2022 00:00
...,...,...,...
355261,658,1,31-10-2022 23:59
355262,582,1,31-10-2022 23:59
355263,272,1,31-10-2022 23:59
355264,563,1,31-10-2022 23:59


In [3]:
gameplay_data['Datetime'] = pd.to_datetime(gameplay_data['Datetime'],format='mixed')

In [4]:
deposit_data = pd.read_csv('D:/Deposit_Data.csv')
deposit_data

,UserID,Datetime,Amount
0,357,01-10-2022 00:03,2000
1,776,01-10-2022 00:03,2500
2,492,01-10-2022 00:06,5000
3,803,01-10-2022 00:07,5000
4,875,01-10-2022 00:09,1500
...,...,...,...
17433,654,31-10-2022 23:57,1200
17434,980,31-10-2022 23:58,200
17435,2,31-10-2022 23:58,40000
17436,612,31-10-2022 23:58,2800


In [5]:
deposit_data['Datetime'] = pd.to_datetime(deposit_data['Datetime'],format='mixed')

In [6]:
withdrawal_data = pd.read_csv('D:/Withdrawal_Data.csv')
withdrawal_data

,UserID,Datetime,Amount
0,190,01-10-2022 00:03,5872
1,159,01-10-2022 00:16,9540
2,164,01-10-2022 00:24,815
3,946,01-10-2022 00:29,23000
4,763,01-10-2022 00:40,9473
...,...,...,...
3561,559,31-10-2022 23:27,5000
3562,407,31-10-2022 23:51,3000
3563,389,31-10-2022 23:56,14481
3564,11,31-10-2022 23:57,4000


In [7]:
withdrawal_data['Datetime'] = pd.to_datetime(withdrawal_data['Datetime'],format='mixed')

# Part-A

## Part-A1: Loyalty Points Calculation

## Description
    In this section, we calculate loyalty points for each user based on gameplay, deposits, and withdrawals, applying the formula provided in Part A.

### User Data Processing
    i) Load and explore the gameplay, deposit, and withdrawal datasets.
    ii) Extract daily gameplay, deposit, and withdrawal transactions to compute loyalty    points by time slot.

### Loyalty Points Calculation Formula
    The formula for loyalty points calculation is as follows:

### Loyalty Points=(0.01×Deposit Amount)+(0.005×Withdrawal Amount)+(0.001×max(deposit−withdrawal,0))+(0.2×Number of Games Played)
    For this report, we’ll calculate these points based on specific slots and days as specified.

In [8]:
# Define slot timings
slot_s1_start, slot_s1_end = "00:00", "12:00"
slot_s2_start, slot_s2_end = "12:00", "23:59"

# Helper function to filter data by slot on a specific date
def filter_data_by_slot(data, date, slot_start, slot_end):
    start_time = pd.to_datetime(f"{date} {slot_start}")
    end_time = pd.to_datetime(f"{date} {slot_end}")
    return data[(data['Datetime'] >= start_time) & (data['Datetime'] <= end_time)]

# Calculate loyalty points for a specific slot on a specific date
def calculate_slot_points(date, slot_start, slot_end):
    # Filter data for the slot and date
    gameplay_slot = filter_data_by_slot(gameplay_data, date, slot_start, slot_end)
    deposit_slot = filter_data_by_slot(deposit_data, date, slot_start, slot_end)
    withdrawal_slot = filter_data_by_slot(withdrawal_data, date, slot_start, slot_end)
    
    # Calculate points per user
    user_points = {}
    for user_id in gameplay_slot["UserID"].unique():
        games_played = gameplay_slot[gameplay_slot["UserID"] == user_id]["GamesPlayed"].sum()
        deposit_amount = deposit_slot[deposit_slot["UserID"] == user_id]["Amount"].sum()
        withdrawal_amount = withdrawal_slot[withdrawal_slot["UserID"] == user_id]["Amount"].sum()
        deposit_count = deposit_slot[deposit_slot["UserID"] == user_id]["UserID"].count()
        withdrawal_count = withdrawal_slot[withdrawal_slot["UserID"] == user_id]["UserID"].count()

        # Apply the loyalty points formula
        loyalty_points = (
            (0.01 * deposit_amount) + 
            (0.005 * withdrawal_amount) + 
            (0.001 * max(deposit_count - withdrawal_count, 0)) + 
            (0.2 * games_played)
        )
        user_points[user_id] = loyalty_points

    return user_points

# Calculate playerwise points for specified slots
specified_slots = [
    ("2022-10-02", slot_s1_start, slot_s1_end),
    ("2022-10-16", slot_s2_start, slot_s2_end),
    ("2022-10-18", slot_s1_start, slot_s1_end),
    ("2022-10-26", slot_s2_start, slot_s2_end)
]

# Dictionary to hold loyalty points for each slot
slot_points = {}
for date, start, end in specified_slots:
    slot_points[(date, start, end)] = calculate_slot_points(date, start, end)

# Display loyalty points for each slot
for slot, points in slot_points.items():
    print(f"\nLoyalty points for {slot}:")
    for user_id, points in points.items():
        print(f"User {user_id}: {points}")



Loyalty points for ('2022-10-02', '00:00', '12:00'):

Loyalty points for ('2022-10-16', '12:00', '23:59'):
User 442: 4.6000000000000005
User 460: 3.4000000000000004
User 524: 1.4000000000000001
User 663: 130.001
User 843: 2.2
User 535: 1.6
User 618: 137.70100000000002
User 722: 112.40100000000001
User 85: 95.603
User 449: 22.400000000000002
User 775: 0.8
User 296: 64.18
User 352: 41.201
User 765: 77.60000000000001
User 181: 8.4
User 797: 1.0
User 189: 0.6000000000000001
User 582: 13.4
User 611: 3.2
User 687: 141.721
User 233: 4.800000000000001
User 377: 191.615
User 941: 1.8
User 710: 11.200000000000001
User 458: 15.600999999999999
User 205: 12.4
User 730: 1.6
User 887: 14.200000000000001
User 856: 19.200000000000003
User 990: 12.4
User 832: 0.6000000000000001
User 754: 122.485
User 632: 10.600999999999999
User 505: 6.800000000000001
User 827: 0.2
User 748: 7.6000000000000005
User 651: 2.6
User 851: 22.201
User 407: 190.9
User 758: 1.0
User 734: 0.8
User 673: 90.80199999999999
User 36

## Part-A2 Monthly Leaderboard Ranking
### Desrciption
    In this section, we calculate the overall loyalty points earned by each user for October, rank users based on their total points, and apply tie-breaking rules.

In [9]:
# Part 2: Calculate total loyalty points for October and rank players
october_points = {}

# Aggregate points across all slots in October
for date, start, end in specified_slots:
    points = calculate_slot_points(date, start, end)
    for user_id, point in points.items():
        if user_id in october_points:
            october_points[user_id] += point
        else:
            october_points[user_id] = point

# Create a DataFrame for ranking
october_points_df = pd.DataFrame.from_dict(october_points, orient='index', columns=['Loyalty Points'])
october_points_df['GamesPlayed'] = gameplay_data.groupby('UserID')['GamesPlayed'].sum()

# Rank by loyalty points, breaking ties by games played
october_points_df = october_points_df.sort_values(by=['Loyalty Points', 'GamesPlayed'], ascending=[False, False])

october_points_df.head(50)

,Loyalty Points,GamesPlayed
369,2002.116,37
208,1704.601,507
634,1237.010,24
538,1200.603,29
673,993.603,372
587,932.202,734
455,904.404,374
566,882.003,183
352,845.560,313
547,716.605,1248


### Part-A3 Average Deposit Amount

### Desrciption
    In this section, we calculate the average deposit amount

In [11]:
# Average deposit amount
average_deposit = deposit_data['Amount'].mean()
print("\nAverage Deposit Amount:", average_deposit)


Average Deposit Amount: 5492.185399701801


### Part-A4 Average Deposit Amount per user

### Desrciption
    In this section, we calculate the average deposit amount per user in October Month

In [10]:
# Average deposit amount per user
average_deposit_per_user = deposit_data.groupby('UserID')['Amount'].mean().mean()
print("Average Deposit Amount per User in October:", average_deposit_per_user)

Average Deposit Amount per User in October: 6900.275174462218


### Part-A5 Average number of games played per user

### Desrciption
    In this section, we calculate the average deposit amount

In [33]:
# Average number of games played per user
average_games_per_user = gameplay_data.groupby('UserID')['GamesPlayed'].sum().mean()
print("Average Number of Games Played per User:", average_games_per_user)

Average Number of Games Played per User: 355.267


# Part-B

### Bonus Allocation Strategy
## Description
    Discuss the strategy for bonus allocation and implement it in code.

In [13]:
gameplay_data['Datetime'] = pd.to_datetime(gameplay_data['Datetime'])
deposit_data['Datetime'] = pd.to_datetime(deposit_data['Datetime'])
withdrawal_data['Datetime'] = pd.to_datetime(withdrawal_data['Datetime'])

In [15]:
october_gameplay = gameplay_data[(gameplay_data['Datetime'].dt.month == 10)]
october_deposit = deposit_data[(deposit_data['Datetime'].dt.month == 10)]
october_withdrawal = withdrawal_data[(withdrawal_data['Datetime'].dt.month == 10)]

In [16]:
# Step 3: Calculate loyalty points for each player
def calculate_loyalty_points(user_id, games_played, total_deposit, total_withdrawal, num_deposits, num_withdrawals):
    # Applying the given loyalty point formula
    loyalty_points = (
        0.01 * total_deposit +
        0.005 * total_withdrawal +
        0.001 * max(num_deposits - num_withdrawals, 0) +
        0.2 * games_played
    )
    return loyalty_points

In [17]:
# Step 4: Aggregate data per user
# Calculate total deposits and withdrawals for each user
deposit_summary = october_deposit.groupby('UserID')['Amount'].sum().reset_index()
deposit_summary.columns = ['UserID', 'Total Deposit']
num_deposits = october_deposit.groupby('UserID').size().reset_index(name='Num Deposits')

withdrawal_summary = october_withdrawal.groupby('UserID')['Amount'].sum().reset_index()
withdrawal_summary.columns = ['UserID', 'Total Withdrawal']
num_withdrawals = october_withdrawal.groupby('UserID').size().reset_index(name='Num Withdrawals')

In [18]:
# Calculate total games played
games_played_summary = october_gameplay.groupby('UserID')['GamesPlayed'].sum().reset_index()

In [19]:
#Merge data into a single DataFrame for loyalty points calculation
user_data = pd.merge(games_played_summary, deposit_summary, on='UserID', how='left')
user_data = pd.merge(user_data, withdrawal_summary, on='UserID', how='left')
user_data = pd.merge(user_data, num_deposits, on='UserID', how='left')
user_data = pd.merge(user_data, num_withdrawals, on='UserID', how='left')

In [20]:
# Fill NaN values with 0 for users without any deposits or withdrawals
user_data = user_data.fillna(0)

In [23]:
# Calculate loyalty points for each user
user_data['Loyalty Points'] = user_data.apply(
    lambda row: calculate_loyalty_points(
        row['UserID'],
        row['GamesPlayed'],
        row['Total Deposit'],
        row['Total Withdrawal'],
        row['Num Deposits'],
        row['Num Withdrawals']
    ),
    axis=1
)

In [25]:
# Step 5: Rank users and find top 50
user_data = user_data.sort_values(by=['Loyalty Points', 'GamesPlayed'], ascending=[False, False]).reset_index(drop=True)
top_50 = user_data.head(50)

In [26]:
# Step 6: Bonus Allocation
# Define the bonus pool
total_bonus_pool = 50000
total_loyalty_points_top_50 = top_50['Loyalty Points'].sum()

In [28]:
# Using .loc to assign values to a column safely
top_50.loc[:, 'Bonus'] = top_50['Loyalty Points'] / total_loyalty_points_top_50 * total_bonus_pool


In [31]:
# Step 7: Output results
print("Top 50 players with allocated bonus:")
top_50[['UserID', 'Loyalty Points', 'GamesPlayed', 'Bonus']]

Top 50 players with allocated bonus:


,UserID,Loyalty Points,GamesPlayed,Bonus
0,634,61121.160,22,7512.773254
1,714,14790.824,4,1818.030073
2,672,13238.624,8,1627.239737
3,99,12469.948,4,1532.757098
4,566,12121.382,106,1489.912732
5,369,11137.111,22,1368.930002
6,740,11006.886,2,1352.923255
7,30,10802.561,9,1327.808428
8,365,10389.775,2368,1277.070392
9,569,9942.818,25,1222.132190


# Part-C Evaluation and Recommendations for Formula Improvement
    The current loyalty point formula has a reasonable foundation but could benefit from adjustments to ensure fairness and better reflect player engagement and financial contributions. Here’s an analysis of the formula’s strengths and some areas for improvement.

### Evaluation of Fairness
    1) Rewarding Engagement: The formula factors in the number of games played, encouraging players to stay active.
    2) Rewarding Deposits and Withdrawals: By including deposits and withdrawals, the formula acknowledges financial contributions to the platform.
    3) Encouraging More Deposits: Awarding points based on the difference between deposits and withdrawals motivates players to keep funds within the platform.

### Proposed Adjustments to the Loyalty Point Formula
    1) Increase the weight for deposits to 0.015 and for withdrawals to 0.0075.
    2) Reduce the weight for games played slightly, perhaps to 0.15, so that financial activity and gameplay contribute more evenly.
.

### A possible new formula could look like this:
## Loyalty Points=(0.015×Deposit Amount)+(0.0075×Withdrawal Amount)+(0.001×max(deposit−withdrawal)^2)+(0.15×Number of Games Played×Game Type Multiplier)

## Conclusion
    The current formula is a good start, but adjusting weights, accounting for game type, and incorporating a consistency bonus would make it fairer and better reflect player loyalty. These adjustments could encourage both financial engagement and frequent gameplay, strengthening player retention and loyalty.